In [2]:
import torch 
import math

In [9]:
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

p = torch.tensor([1, 2, 3]) # Polynomial order
xx = x.unsqueeze(-1).pow(p) # X vals before weights

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1), # Weights for order p?
    torch.nn.Flatten(0, 1) # ? 
)
loss_fn = torch.nn.MSELoss(reduction='sum')

In [14]:
learning_rate = 1e-03
optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate)
for t in range(2000):
    y_pred = model(xx)
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99: 
        print(t, loss.item())
        
    optimizer.zero_grad()
    
    loss.backward()
    optimizer.step()

linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} xx^2 + {linear_layer.weight[:, 2].item()} x^3')

99 149.72117614746094
199 74.60148620605469
299 37.8614387512207
399 19.546300888061523
499 11.581292152404785
599 9.221049308776855
699 8.92163372039795
799 8.919583320617676
899 8.90119743347168
999 8.901711463928223
1099 8.909024238586426
1199 8.911714553833008
1299 8.912036895751953
1399 8.912121772766113
1499 8.917753219604492
1599 8.924635887145996
1699 8.921676635742188
1799 8.919417381286621
1899 8.920443534851074
1999 8.92129135131836
Result: y = -0.0005094200605526567 + 0.8572409749031067 x + -0.0005094226216897368 xx^2 + -0.09283032268285751 x^3
